In [1]:
from src.odes        import ODE
from scipy.integrate import solve_ivp

In [ ]:
def ODE(t, )


def ODE(t,Y, args):

    TOTAL = args[0]
    K     = args[1]
    HNR   = args[2]
    ACCR  = args[3]
    
    YDOT  = np.empty(number_of_non_conserved_species)
    X     = np.empty(number_of_conserved_species)

    ...

    return YDOT